

<div style="background-color:#D8D8C6">

    

### <i>Assignment Week 3</i> 

<hr>

## <font color='blue'>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto </font>

<hr>
    
</div>

<div style="background-color:lightgrey">
<br>
    
## Table of Contents (click to navigate)
            
### - [**Task 1:** Create a Datafram which contains Torontos Postal Codes](#T1)
### - [**Task 2:** Add Geocodes (Longitude,Latitude) to the Dataframe](#T2)
### - [**Task 3:** Explore the Neighborhoods get the Venues from Foursquare, visualize on maps and create a cluster (k-means) ](#T3)

<br><br>    
</div>

<hr>
<a id='T1'></a>

<div style="background-color:powderblue">
<br><br> 

## Task 1: Transform Torontos Postal Codes  into a Pandas Dataframe

<br><br> 
</div>
<hr>

In [2]:
import pandas as pd

In [3]:
dfl=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') # Pandas read_html can read html tables
print('Df has type',type(dfl))  # we get a list back which contains dataframes as elements
for i,d in enumerate(dfl):
    print ('Element {} contains {} rows and {} columns'.format(i, d.shape[0],d.shape[1]))

Df has type <class 'list'>
Element 0 contains 180 rows and 3 columns
Element 1 contains 4 rows and 18 columns
Element 2 contains 2 rows and 18 columns


<hr>

### Element 0 contains the  table of interest. 
### ==> change the column names according Assignment instructions

<hr>

In [4]:
print ("Original Columns names: ", dfl[0].columns)
# renaming according Assignment instructions:
dfl[0].rename(columns={'Postal code':'PostalCode'}, inplace=True)
#assign the  table  of interest to df
df=dfl[0]
df.head()

Original Columns names:  Index(['Postal code', 'Borough', 'Neighborhood'], dtype='object')


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


<hr>

### Get an overview  how many Boroughs are "Not assigned" and check for other noticeable items 

<hr>

In [5]:
print (df['Borough'].value_counts())
print (df.shape)

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64
(180, 3)


<hr>

### Remove rows where Borough is "Not assigned"

<hr>

In [6]:
df=df[df['Borough']!="Not assigned"]   # remove rows which contains Boroughs named "not assigned"
df.shape

(103, 3)

<hr>

### The Overall Rows are now 103 after we got rid of the "Not assigned" rows

<hr>

In [7]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


<hr>

### Now we check for duplicated Postal Codes

<hr>

In [8]:
#Check for duplicated Neighborhood Names
df[df['PostalCode'].duplicated(keep=False)].shape

(0, 3)

<hr>

### As the shape output shows, there is NO duplicated Postal Code although indicated by the Assignment instructions. Probably the table format has changed meanwhile.
### The duplicated PostalCodes are already merged at the Website. The delimiters are not "," but "/" .  Lets list those and replace "/" by "," as "," was originally requested by the Assignment Instructions

<hr>

In [9]:
df[df['Neighborhood'].str.contains("/")].head()

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
9,M1B,Scarborough,Malvern / Rouge
12,M4B,East York,Parkview Hill / Woodbine Gardens


In [10]:
df['Neighborhood']=df['Neighborhood'].str.replace("/",",")
df[df['Neighborhood'].str.contains(",")].head()


C:\Users\heinz\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
9,M1B,Scarborough,"Malvern , Rouge"
12,M4B,East York,"Parkview Hill , Woodbine Gardens"


<hr>

### The dataframe is now correctly wrangled and formatted

<hr>

In [11]:
print ("The final Table contains {} rows and {} columns".format(df.shape[0],df.shape[1]))
print ("shape output:",df.shape)

The final Table contains 103 rows and 3 columns
shape output: (103, 3)


<hr>

## Result Task1:  The Shape output shows (103,3)

<hr>

<a id='T2'></a>
<div style="background-color:powderblue">
<br>
    
    
# **Task 2:** Get the Geo Location for each neighborhood

==> I was not successful to get any geo positions (lat/long) by geocoding the Three-Letter postalcode.  Neither with google ("access denied") nor with Nominatim (just did not recognize the 3 letter codes) <br>
==> Maybe this is because the 3letters are not sufficient to identify the neighborhoods. The listed codes are only the outwards codes so Nominatim probably needs the whole postal/zip codes...  <br>
==> After spending hours of searching around I eventually decided to use the provided CSV <br>

<br>
</div>    
<hr>

In [12]:
#!conda install -c conda-forge folium --yes
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge geocoder --yes
import pandas as pd

In [13]:

dfloc=pd.read_csv('http://cocl.us/Geospatial_data')
dfloc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [14]:
dfloc.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
dfloc.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
#Check for duplicated postal codes just to be sure the data is ok
dfloc[dfloc['PostalCode'].duplicated(keep=False)].shape

(0, 3)

In [16]:
# Merge both dataframes via left join (similar to sql join). Also tested with "inner" join => same result
dfmerge=pd.merge(df,dfloc,on='PostalCode',how='left')

In [17]:
dfmerge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [18]:
dfmerge.shape

(103, 5)

<hr>

## Result Task2:  See the Dataframe head above.  Latitude and Longitude are included now.

<hr>

<a id='T3'></a>
<div style="background-color:powderblue">
<br>
    
    
# **Task 3:** Explore the Neighborhoods get the Venues from Foursquare, visualize on maps and create a cluster (k-means)
    ==> I chose similar method/coding as listed and practiced  in the lab. 
    ==> The venue request to Foursquare is based on the latitude/longitude data coming with the CSV file from coursera.
    ==> Accordingly the requests and results to Foursquare contain the venues around the lat/long values for the 3 letter Postalcode which in most cases represent one neighborhood.
    ==> In some cases multiple neighborhoods are consolidated to one row as they belong to one 3 letter postalCode.


<br>
</div>    
<hr>

### Method
- use geopy.geocoders to get the central latitude/longitude coordinates which is used to centralize the starting Folium view for Toronto.
- draw a first folium map with the centroids of the neighborhoods containing the dataframe created at Task2. The dataframe is called "dfmerge"
- run forsquare request to get the venues (max 100 venues per Neighborhood, limited to 500m radius). The data of interest is listed in the column 'venue categories'
- Do a so called onehot transformation with the column 'venue categories' so we have every category as column headlines. The result is stored in Dataframe 'nearby_venues_onehot'
- Now we have for each venue a separate row. In order to get a ranking we need to derive the fequency number. This is realized by grouping the neighborhood column while building the mean values of the venue categories columns. The result is stored at Dataframe "nearby_venues.groupby"
- Get the top 5 ranking of venue categories for each neighborhood
- run sklearn k-means cluster tool. I chose 5 cluster categories to be calculated. The cluster classification is applied on the neighborhoods, not on the venues.
- Visualize the cluster categories by creating another folium map with same cluster points as before but with colorized cluster categories.
- The cluster items are then reported for each cluster category



In [19]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [20]:
#get the central coordinates of Toronto
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


In [21]:
# draw a basic folium map including the postal code locations

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfmerge['Latitude'], dfmerge['Longitude'], dfmerge['Borough'], dfmerge['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [22]:
#### the foursquare client id and secret have been removed for security reasons####
CLIENT_ID = 'putyourclientid here' # your Foursquare ID
CLIENT_SECRET = 'putyoursecredhere' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XBJ1BHFPVSMDTQUR5PUNEAES2JPKNYQU3FRR0YLHJT2UG3R5
CLIENT_SECRET:ASMBYTIKO1BBYZTV5OW1CQFUMTIMPBSGBDRM2ZWDDTSMJXPJ


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, lat, lng)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print (url)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
nearby_venues=getNearbyVenues(names=dfmerge['Neighborhood'],latitudes=dfmerge['Latitude'],longitudes=dfmerge['Longitude'])

Parkwoods 43.7532586 -79.3296565
Victoria Village 43.725882299999995 -79.31557159999998
Regent Park , Harbourfront 43.6542599 -79.3606359
Lawrence Manor , Lawrence Heights 43.718517999999996 -79.46476329999999
Queen's Park , Ontario Provincial Government 43.6623015 -79.3894938
Islington Avenue 43.6678556 -79.53224240000002
Malvern , Rouge 43.806686299999996 -79.19435340000001
Don Mills 43.745905799999996 -79.352188
Parkview Hill , Woodbine Gardens 43.7063972 -79.309937
Garden District, Ryerson 43.6571618 -79.37893709999999
Glencairn 43.709577 -79.44507259999999
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale 43.6509432 -79.55472440000001
Rouge Hill , Port Union , Highland Creek 43.7845351 -79.16049709999999
Don Mills 43.72589970000001 -79.340923
Woodbine Heights 43.695343900000005 -79.3183887
St. James Town 43.6514939 -79.3754179
Humewood-Cedarvale 43.6937813 -79.42819140000002
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood 43.6435152 -79.57

In [25]:
print (nearby_venues.shape)
print (nearby_venues.info())

(2201, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201 entries, 0 to 2200
Data columns (total 7 columns):
Neighborhood              2201 non-null object
Neighborhood Latitude     2201 non-null float64
Neighborhood Longitude    2201 non-null float64
Venue                     2201 non-null object
Venue Latitude            2201 non-null float64
Venue Longitude           2201 non-null float64
Venue Category            2201 non-null object
dtypes: float64(4), object(3)
memory usage: 120.5+ KB
None


In [26]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [27]:
# how many venues are listed in each neighborhood:
pd.set_option('display.max_rows', 200)
nearby_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood , Long Branch",10,10,10,10,10,10
"Bathurst Manor , Wilson Heights , Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55
"Birch Cliff , Cliffside West",4,4,4,4,4,4
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
Business reply mail Processing CentrE,16,16,16,16,16,16


In [28]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 268 uniques categories.


In [29]:

# one hot encoding
nearby_venues_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")
nearby_venues_onehot.columns

# add neighborhood column back to dataframe
nearby_venues_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns=['Neighborhood']+ list(nearby_venues_onehot.columns.drop('Neighborhood'))
                                    
# -- this is the code from the lab but does not work here -- fixed_columns = [nearby_venues_onehot.columns[-1]] + list(nearby_venues_onehot.columns[:-1])
nearby_venues_onehot = nearby_venues_onehot[fixed_columns]

nearby_venues_onehot.head()


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
nearby_venues_onehot.shape

(2201, 268)

In [31]:
neighborhood_grouped = nearby_venues_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Alderwood , Long Branch",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bathurst Manor , Wilson Heights , Downsview North",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bayview Village,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Bedford Park , Lawrence Manor East",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.040000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Berczy Park,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Birch Cliff , Cliffside West",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
9,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.000000,0.062500,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
neighborhood_grouped.shape

(94, 268)

In [33]:
num_top_venues = 5

for hood in neighborhood_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighborhood_grouped[neighborhood_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1  Latin American Restaurant  0.25
2               Skating Rink  0.25
3                     Lounge  0.25
4          Accessories Store  0.00


----Alderwood , Long Branch----
            venue  freq
0     Pizza Place   0.2
1             Pub   0.1
2        Pharmacy   0.1
3  Sandwich Place   0.1
4             Gym   0.1


----Bathurst Manor , Wilson Heights , Downsview North----
            venue  freq
0            Bank  0.11
1     Coffee Shop  0.11
2           Diner  0.05
3   Shopping Mall  0.05
4  Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park , Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.08
1         Pizza Place  0.08
2      Sandwich Place  0.08
3          Restaurant  0.08
4    

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,"Alderwood , Long Branch",Pizza Place,Dance Studio,Pharmacy,Sandwich Place,Pub,Athletics & Sports,Skating Rink,Coffee Shop,Gym,Dim Sum Restaurant
2,"Bathurst Manor , Wilson Heights , Downsview North",Bank,Coffee Shop,Ice Cream Shop,Restaurant,Sushi Restaurant,Deli / Bodega,Shopping Mall,Middle Eastern Restaurant,Diner,Fried Chicken Joint
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,"Bedford Park , Lawrence Manor East",Restaurant,Sandwich Place,Italian Restaurant,Pizza Place,Coffee Shop,Comfort Food Restaurant,Indian Restaurant,Sushi Restaurant,Café,Butcher


In [36]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [37]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfmerge

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Fast Food Restaurant,Dumpling Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Hockey Arena,Intersection,Pizza Place,Coffee Shop,Portuguese Restaurant,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Yoga Studio,Ice Cream Shop
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1.0,Furniture / Home Store,Accessories Store,Clothing Store,Coffee Shop,Event Space,Miscellaneous Shop,Athletics & Sports,Boutique,Vietnamese Restaurant,Creperie
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Yoga Studio,Restaurant,Bar,Beer Bar,Italian Restaurant,Juice Bar,Burger Joint,Burrito Place,Café


In [38]:
#Check for empty neighborhoods which means that there are no foursquare venues available.
toronto_merged[toronto_merged.isnull().any(axis=1)]


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,M2L,North York,"York Mills , Silver Hills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,M2M,North York,"Willowdale , Newtonbrook",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# remove those rows with null/NaN Cluster label because we cannot put them into the cluster folium label.
print ("Shape before NaN row deletion:", toronto_merged.shape)
toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)
# check and change dtype of "Cluster Labels" to integer
print ("Shape after NaN row deletion:", toronto_merged.shape)
toronto_merged['Cluster Labels'].dtypes   #  ==> its float64 so we need to change to in
# change from float to integer
toronto_merged=toronto_merged.astype({'Cluster Labels': 'int'})

Shape before NaN row deletion: (103, 16)
Shape after NaN row deletion: (99, 16)


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Food & Drink Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Fast Food Restaurant,Dumpling Restaurant
21,York,0,Park,Women's Store,Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Diner,Eastern European Restaurant
35,East York,0,Park,Convenience Store,Coffee Shop,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
64,York,0,Park,Convenience Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
66,North York,0,Park,Bank,Convenience Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
68,Central Toronto,0,Park,Sushi Restaurant,Jewelry Store,Trail,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
85,Scarborough,0,Park,Playground,Coffee Shop,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
91,Downtown Toronto,0,Park,Playground,Trail,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Hockey Arena,Intersection,Pizza Place,Coffee Shop,Portuguese Restaurant,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Yoga Studio,Ice Cream Shop
3,North York,1,Furniture / Home Store,Accessories Store,Clothing Store,Coffee Shop,Event Space,Miscellaneous Shop,Athletics & Sports,Boutique,Vietnamese Restaurant,Creperie
4,Downtown Toronto,1,Coffee Shop,Yoga Studio,Restaurant,Bar,Beer Bar,Italian Restaurant,Juice Bar,Burger Joint,Burrito Place,Café
6,Scarborough,1,Fast Food Restaurant,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
7,North York,1,Gym,Coffee Shop,Restaurant,Beer Store,Japanese Restaurant,Supermarket,Clothing Store,Dim Sum Restaurant,Chinese Restaurant,Caribbean Restaurant
8,East York,1,Pizza Place,Gym / Fitness Center,Pharmacy,Intersection,Athletics & Sports,Gastropub,Bus Line,Pet Store,Fast Food Restaurant,Bank
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Café,Restaurant,Bookstore,Ramen Restaurant
10,North York,1,Park,Pizza Place,Pub,Japanese Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
11,Etobicoke,1,Golf Course,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,Etobicoke,2,River,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,3,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
83,Central Toronto,3,Summer Camp,Playground,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4,Baseball Field,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
101,Etobicoke,4,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
